In [1]:
#pip install --no-cache-dir jupyter langchain_openai langchain_community langchain langgraph faiss-cpu sentence-transformers ipywidgets transformers nltk scikit-learn matplotlib markdown langchain_chroma

import yaml
import ipywidgets as widgets
from IPython.display import display
import pickle
import markdown

from story_sage.story_sage import StorySage

In [5]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

api_key = config['OPENAI_API_KEY']
chroma_path = config['CHROMA_PATH']
chroma_collection = config['CHROMA_COLLECTION']
merged_characters_path = config['MERGED_CHARACTERS_PATH']


with open(merged_characters_path, 'rb') as f:
    character_dict = pickle.load(f)

story_sage = StorySage(api_key=api_key, chroma_path=chroma_path, 
                       chroma_collection_name=chroma_collection,
                       character_dict=character_dict, n_chunks=10)

In [6]:
story_sage.invoke(question='where does mat find his spear?')

("Mat finds his spear in a treasure room. Here are some relevant points from the context that support this:\n\n- **Location of the Spear**: Mat mentions, “. . . Mat took it from the treasure room,” indicating that he retrieved the spear from this specific location.\n- **Description of the Spear**: The spear has an unusual design, featuring a two-foot sword blade marked with ravens, which implies it is not an ordinary weapon. This unique quality adds to its significance.\n- **Mat's Initial Reaction**: After finding the spear, Mat expresses skepticism about its reliability, saying he would not trust it unless he had to, despite its long-lasting qualities.\n\nThese points clearly illustrate where Mat found his spear and provide insight into his thoughts and feelings about it.",
 {'ids': [['7a7bf3b7-c283-4463-98b3-eebfde59ffa0',
    '9168b765-34a4-44e2-8461-759a78c21ffe',
    '156ac0b6-2c0e-4ecc-a106-91d951c65f21',
    'c4471b75-1e59-457a-928a-26312a451d10',
    '0d784036-85fe-48bf-a5ea-41

In [3]:
# Define the input and output widgets
input_box = widgets.Text(
    value='',
    placeholder='Type your question here...',
    description='Question:',
    continuous_update=False,
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='',
    tooltip='Click to submit your question',
    icon='check'
)

book_number_box = widgets.IntText(
    value=10,
    description='Book Number:',
    disabled=False
)

chapter_number_box = widgets.IntText(
    value=None,
    description='Chapter Number:',
    disabled=False
)

status_box = widgets.Output(layout={'min_height': '50px'})
output_box = widgets.Output(layout={'min_height': '200px'})
context_box = widgets.Output(layout={'min_height': '200px'})

# Create a spinner widget
spinner = widgets.HTML(
    value="""<i class="fa fa-spinner fa-spin" style="font-size:24px; color:#2a9df4;"></i>""",
    placeholder='Loading...',
    description=''
)

# Make sure Font Awesome is available
display(widgets.HTML("<link rel='stylesheet' href='https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css'>"))

def wrap_answer(answer):
    html = markdown.markdown(answer)
    return f"<div style='background-color: #f9f9f9; padding: 10px; border-radius: 5px;'>{html}</div>"

def show_results(answer, context):
    with output_box:
        output_box_contents = []
        output_box_contents.append("<h3>Answer</h3>")
        output_box_contents.append(wrap_answer(answer))
        display(widgets.HTML(''.join(output_box_contents)))
        with context_box:
            context_box.clear_output()
            context_box_contents = []
            context_box_contents.append("<h3>Context</h3>")
            for idx in range(len(context['metadatas'])):
                meta = context['metadatas'][0][idx]
                content = context['documents'][0][idx]
                context_box_contents.append(f"<p><strong>Book Number:</strong> {meta['book_number']} <strong>Chapter Number:</strong> {meta['chapter_number']}</p>")
                context_box_contents.append(f"<p>{content}</p>")
            display(widgets.HTML(wrap_answer("".join(context_box_contents))))

    

# Define the function to handle the button click
def submit_question(b):
    with status_box:
        status_box.clear_output()
        display(widgets.HTML(f"<h3>Retrieving top relevant chunks...</h3>"))
        with output_box:
            output_box.clear_output()
            display(spinner)
            answer, context = story_sage.invoke(
                question = input_box.value,
                book_number = book_number_box.value,
                chapter_number = chapter_number_box.value
            )
            output_box.clear_output()
    show_results(answer, context)

# Attach the handler to the button
submit_button._click_handlers.callbacks.clear()
submit_button.on_click(submit_question)

# Attach the handler to the input box for the return key
#input_box.observe(submit_question)

# Display the widgets
display(status_box, book_number_box, chapter_number_box, input_box, submit_button, output_box, context_box)

HTML(value="<link rel='stylesheet' href='https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-aw…

Output(layout=Layout(min_height='50px'))

IntText(value=10, description='Book Number:')

IntText(value=0, description='Chapter Number:')

Text(value='', continuous_update=False, description='Question:', placeholder='Type your question here...')

Button(description='Submit', icon='check', style=ButtonStyle(), tooltip='Click to submit your question')

Output(layout=Layout(min_height='200px'))

Output(layout=Layout(min_height='200px'))

UnboundLocalError: cannot access local variable 'answer' where it is not associated with a value

In [4]:
story_sage.state

{}